In [27]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

In [28]:
# load data
Data = pd.read_csv("Life Expectancy Data.csv")
cols = Data.columns
print(Data.columns)
print(Data)

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'Alcohol', 'percentage expenditure', 'Hepatitis B',
       'Measles ', ' BMI ', 'under-five deaths ', 'Polio', 'Total expenditure',
       'Diphtheria ', ' HIV/AIDS', 'GDP', 'Population',
       ' thinness  1-19 years', ' thinness 5-9 years',
       'Income composition of resources', 'Schooling'],
      dtype='object')
          Country  Year      Status  Life expectancy   Adult Mortality  \
0     Afghanistan  2015  Developing              65.0            263.0   
1     Afghanistan  2014  Developing              59.9            271.0   
2     Afghanistan  2013  Developing              59.9            268.0   
3     Afghanistan  2012  Developing              59.5            272.0   
4     Afghanistan  2011  Developing              59.2            275.0   
...           ...   ...         ...               ...              ...   
2933     Zimbabwe  2004  Developing              44.3         

In [29]:
# Find the percentage of None data for each colume
Data.isna().sum()/len(Data)

Country                            0.000000
Year                               0.000000
Status                             0.000000
Life expectancy                    0.003404
Adult Mortality                    0.003404
infant deaths                      0.000000
Alcohol                            0.066031
percentage expenditure             0.000000
Hepatitis B                        0.188223
Measles                            0.000000
 BMI                               0.011572
under-five deaths                  0.000000
Polio                              0.006467
Total expenditure                  0.076923
Diphtheria                         0.006467
 HIV/AIDS                          0.000000
GDP                                0.152485
Population                         0.221920
 thinness  1-19 years              0.011572
 thinness 5-9 years                0.011572
Income composition of resources    0.056841
Schooling                          0.055480
dtype: float64

In [30]:
# Drop rows that has no "live expectancy" column since it is just 0.003404
Data = Data[~Data["Life expectancy "].isna()]

# Drop rows that are missing less than 5% of data 
non_perc = (Data.isna().sum()/len(Data)).values
less5Cols = non_perc < 0.05
print(cols[less5Cols])
for col in cols[less5Cols]:
    if Data[col].isna().sum()/len(Data) > 1e-3:
        print(col)
        print("before :", len(Data))
        Data = Data[~Data[col].isna()]
        print("after :",len(Data))   

print()
print("After droping")
print(Data.isna().sum()/len(Data)) 

Index(['Country', 'Year', 'Status', 'Life expectancy ', 'Adult Mortality',
       'infant deaths', 'percentage expenditure', 'Measles ', ' BMI ',
       'under-five deaths ', 'Polio', 'Diphtheria ', ' HIV/AIDS',
       ' thinness  1-19 years', ' thinness 5-9 years'],
      dtype='object')
 BMI 
before : 2928
after : 2896
Polio
before : 2896
after : 2888

After droping
Country                            0.000000
Year                               0.000000
Status                             0.000000
Life expectancy                    0.000000
Adult Mortality                    0.000000
infant deaths                      0.000000
Alcohol                            0.060596
percentage expenditure             0.000000
Hepatitis B                        0.181787
Measles                            0.000000
 BMI                               0.000000
under-five deaths                  0.000000
Polio                              0.000000
Total expenditure                  0.073407
Diphtheria   